In [1]:
from collections import defaultdict
from datetime import datetime, timedelta
import os
from copy import deepcopy
import yaml
import functools

from clickhouse_driver import Client


from google_sheet import write_df, write_update_time, write_data_to_gs_no_resize
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import pygsheets
import scipy
from scipy import stats


from datetime import datetime, timedelta
import time

import logging

In [2]:
def read_yaml_config(file_name: str) -> any:
    with open(file_name) as f:
        return yaml.safe_load(f)

def get_config(
        path_dir_main_config: str,
        path_sensitive_config: str
):
    path_main_config = os.path.join(path_dir_main_config, 'config.yaml')
    path_input_csv = os.path.join(path_dir_main_config, 'input.csv')
    
    with open(path_main_config,'r') as stream:
        main_config = yaml.safe_load(stream)

    with open(path_sensitive_config,'r') as stream:
        conf = yaml.safe_load(stream)
        cli_creds = conf['clickhouse']
        cg_api_key = conf['cg-api-key']
    
    input_p = pd.read_csv(path_input_csv)
    input_p = input_p.set_index('coingecko_id')
    return input_p, main_config, cli_creds, cg_api_key


def setup_logging():
    file_path = os.path.dirname(__file__)
    logging.basicConfig(
        filename=os.path.join(file_path, "main.log"),
        filemode='a',
        level=logging.INFO,
        format='%(asctime)s %(levelname)-8s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
def get_enter_prices(input_c):
    buy_prices_usdt = input_p['price_USDT'].to_dict()
    buy_prices_btc = input_p['price_BTC'].to_dict()
    buy_prices_eth = input_p['price_ETH'].to_dict()
    return buy_prices_usdt, buy_prices_btc, buy_prices_eth

def get_info(input_c):
    info = input_p[['ticker', 'price_USDT', 'price_BTC', 'price_ETH', 'amount_usdt', 'dt']] #.set_index('ticker')
    return info

In [3]:
input_p, main_config, cli_creds, cg_api_key = get_config('configs/config_5_25_03_2023/', 'configs/sensitive.yaml')

cli = Client(
    host=main_config['clickhouse']['client']['host'],
    user=cli_creds['user'],
    password=cli_creds['password']
        #settings=main_config['clickhouse']['client']['settings']
    )

cli.execute('select now();')


[(datetime.datetime(2023, 5, 27, 6, 32, 49),)]

In [28]:
def task_step_day(dt_close, cli, conf):
    clii = DataInterfaceDynamic(cli, conf, dt_close, cli_external=cli)
    coins_track = clii.get_inter_coins_today()
    
    prices_wrt_btc = clii.get_prices_wrt_base_dt_close(coins_track, 'bitcoin')
    prices_wrt_eth = clii.get_prices_wrt_base_dt_close(coins_track, 'ethereum')
    prices_wrt_usd = clii.get_prices_wrt_base_dt_close(coins_track, 'usdt')
    
    prices = pd.merge(
        pd.merge(
            prices_wrt_btc, prices_wrt_eth, on=['coingecko_id','dt_candle_close']
        ), 
        prices_wrt_usd, on=['coingecko_id','dt_candle_close' ]
    )
    
    btc_sum_today = clii.get_btc_sum(coins_track)
    coins_today = determine_coins_today(coins_track, btc_sum_today)
    realized, coins_yesterday, unrealized = clii.get_dynaimic_state()
    logging.warning(f'Get dynamic state')
    logging.warning(f'Realized history {len(realized)}, Unrealized history {len(unrealized)}')
    
    unrealized, realized_today, snapshot, portfolio_members = (
        step_day(dt_close, coins_today, coins_yesterday, unrealized, realized, prices)
    )
    
    clii.put_today_realized(realized_today)
    clii.put_unrealized(unrealized)
    clii.put_snapshot(snapshot)
    clii.put_portfolio_coins(portfolio_members)

In [34]:
task_metric_by_coins('beta_bitcoin', dt_close, cli, conf)
task_metric_by_coins('corr_bitcoin', dt_close, cli, conf)

task_metric_by_coins('beta_ethereum', dt_close, cli, conf)
task_metric_by_coins('corr_ethereum', dt_close, cli, conf)

['radiant-capital', 'okb', 'gamma-strategies', 'injective-protocol', 'floki', 'spool-dao-token', 'nakamoto-games', 'kaspa', 'pip', 'highstreet', 'winr-protocol']


['radiant-capital', 'okb', 'gamma-strategies', 'injective-protocol', 'floki', 'spool-dao-token', 'nakamoto-games', 'kaspa', 'pip', 'highstreet', 'winr-protocol']


['radiant-capital', 'okb', 'gamma-strategies', 'injective-protocol', 'floki', 'spool-dao-token', 'nakamoto-games', 'kaspa', 'pip', 'highstreet', 'winr-protocol']


['radiant-capital', 'okb', 'gamma-strategies', 'injective-protocol', 'floki', 'spool-dao-token', 'nakamoto-games', 'kaspa', 'pip', 'highstreet', 'winr-protocol']


In [42]:
dt_close = '2023-05-25'
conf = ConfigInterfaceDynamic('configs/conf_new.yaml')


In [43]:
task_step_day(dt_close, cli, conf)

floki


,coingecko_id,dt_candle_close,dt_in,price_in_usd,price_in_btc,price_in_eth,usd_in,btc_in,eth_in
coingecko_id,,,,,,,,,
radiant-capital 2023-03-25,radiant-capital,2023-05-24,2023-03-25,0.339843,1.228191e-05,1.937551e-04,10000.0,10000.0,10000.0
okb 2023-04-14,okb,2023-05-24,2023-04-14,49.850964,1.636153e-03,2.370530e-02,10000.0,10000.0,10000.0
gamma-strategies 2023-04-20,gamma-strategies,2023-05-24,2023-04-20,0.314238,1.112126e-05,1.616919e-04,10000.0,10000.0,10000.0
injective-protocol 2023-04-20,injective-protocol,2023-05-24,2023-04-20,7.898362,2.795328e-04,4.064124e-03,10000.0,10000.0,10000.0
floki 2023-04-24,floki,2023-05-24,2023-04-24,0.000045,1.619338e-09,2.417774e-08,10000.0,10000.0,10000.0
spool-dao-token 2023-05-05,spool-dao-token,2023-05-24,2023-05-05,0.910145,3.083115e-05,4.565307e-04,10000.0,10000.0,10000.0
nakamoto-games 2023-05-19,nakamoto-games,2023-05-24,2023-05-19,0.237491,8.833777e-06,1.310559e-04,10000.0,10000.0,10000.0
kaspa 2023-05-24,kaspa,2023-05-24,2023-05-24,0.016625,6.312086e-07,9.232542e-06,10000.0,10000.0,10000.0
winr-protocol 2023-05-24,winr-protocol,2023-05-24,2023-05-24,0.050893,1.932233e-06,2.826232e-05,10000.0,10000.0,10000.0


gamma-strategies


,coingecko_id,dt_candle_close,dt_in,price_in_usd,price_in_btc,price_in_eth,usd_in,btc_in,eth_in
coingecko_id,,,,,,,,,
radiant-capital 2023-03-25,radiant-capital,2023-05-24,2023-03-25,0.339843,1.228191e-05,0.000194,10000.0,10000.0,10000.0
okb 2023-04-14,okb,2023-05-24,2023-04-14,49.850964,1.636153e-03,0.023705,10000.0,10000.0,10000.0
gamma-strategies 2023-04-20,gamma-strategies,2023-05-24,2023-04-20,0.314238,1.112126e-05,0.000162,10000.0,10000.0,10000.0
injective-protocol 2023-04-20,injective-protocol,2023-05-24,2023-04-20,7.898362,2.795328e-04,0.004064,10000.0,10000.0,10000.0
spool-dao-token 2023-05-05,spool-dao-token,2023-05-24,2023-05-05,0.910145,3.083115e-05,0.000457,10000.0,10000.0,10000.0
nakamoto-games 2023-05-19,nakamoto-games,2023-05-24,2023-05-19,0.237491,8.833777e-06,0.000131,10000.0,10000.0,10000.0
kaspa 2023-05-24,kaspa,2023-05-24,2023-05-24,0.016625,6.312086e-07,0.000009,10000.0,10000.0,10000.0
winr-protocol 2023-05-24,winr-protocol,2023-05-24,2023-05-24,0.050893,1.932233e-06,0.000028,10000.0,10000.0,10000.0


injective-protocol


...

,coingecko_id,dt_candle_close,dt_in,price_in_usd,price_in_btc,price_in_eth,usd_in,btc_in,eth_in
coingecko_id,,,,,,,,,
radiant-capital 2023-03-25,radiant-capital,2023-05-24,2023-03-25,0.339843,0.000012,0.000194,10000.0,10000.0,10000.0
spool-dao-token 2023-05-05,spool-dao-token,2023-05-24,2023-05-05,0.910145,0.000031,0.000457,10000.0,10000.0,10000.0
winr-protocol 2023-05-24,winr-protocol,2023-05-24,2023-05-24,0.050893,0.000002,0.000028,10000.0,10000.0,10000.0


spool-dao-token


,coingecko_id,dt_candle_close,dt_in,price_in_usd,price_in_btc,price_in_eth,usd_in,btc_in,eth_in
coingecko_id,,,,,,,,,
spool-dao-token 2023-05-05,spool-dao-token,2023-05-24,2023-05-05,0.910145,0.000031,0.000457,10000.0,10000.0,10000.0
winr-protocol 2023-05-24,winr-protocol,2023-05-24,2023-05-24,0.050893,0.000002,0.000028,10000.0,10000.0,10000.0


winr-protocol


,coingecko_id,dt_candle_close,dt_in,price_in_usd,price_in_btc,price_in_eth,usd_in,btc_in,eth_in
coingecko_id,,,,,,,,,
radiant-capital 2023-03-25,radiant-capital,2023-05-24,2023-03-25,0.339843,1.228191e-05,1.937551e-04,10000.0,10000.0,10000.0
okb 2023-04-14,okb,2023-05-24,2023-04-14,49.850964,1.636153e-03,2.370530e-02,10000.0,10000.0,10000.0
gamma-strategies 2023-04-20,gamma-strategies,2023-05-24,2023-04-20,0.314238,1.112126e-05,1.616919e-04,10000.0,10000.0,10000.0
injective-protocol 2023-04-20,injective-protocol,2023-05-24,2023-04-20,7.898362,2.795328e-04,4.064124e-03,10000.0,10000.0,10000.0
floki 2023-04-24,floki,2023-05-24,2023-04-24,0.000045,1.619338e-09,2.417774e-08,10000.0,10000.0,10000.0
spool-dao-token 2023-05-05,spool-dao-token,2023-05-24,2023-05-05,0.910145,3.083115e-05,4.565307e-04,10000.0,10000.0,10000.0
nakamoto-games 2023-05-19,nakamoto-games,2023-05-24,2023-05-19,0.237491,8.833777e-06,1.310559e-04,10000.0,10000.0,10000.0
kaspa 2023-05-24,kaspa,2023-05-24,2023-05-24,0.016625,6.312086e-07,9.232542e-06,10000.0,10000.0,10000.0
winr-protocol 2023-05-24,winr-protocol,2023-05-24,2023-05-24,0.050893,1.932233e-06,2.826232e-05,10000.0,10000.0,10000.0


In [12]:
#beta, corr
def task_metric_by_coins(metric, dt_close, cli, conf):
    logging.info(f'Start calc {metric} at {dt_close}')
    clii = DataInterfaceDynamic(cli, conf, dt_close, cli_external=cli)
    smc = SingleMetricCalculator(dt_close, clii, conf)
    coins_metric = smc.get_metric_by_coins(metric)
    clii.put_metric_dt_candle_close(metric, coins_metric)

In [13]:
def task_total_by_metric(metric, dt_close, cli, conf):
    clii = DataInterfaceDynamic(cli, conf, dt_close)
    gse = GoogleSheetExporter(conf, dt_close)
    tc = TotalCalculatorDynamic(dt_close, clii)
    
    value = tc.get_total_by_day(metric)
    clii.put_total_metric(metric, value)
    
    gse.put_metric_total_day(metric, value)
    logging.info(f'Total {metric} written to cli & gsh')

In [102]:
task_total_by_metric('beta_bitcoin', dt_close, cli, conf)
task_total_by_metric('corr_bitcoin', dt_close, cli, conf)
task_total_by_metric('beta_ethereum', dt_close, cli, conf)
task_total_by_metric('corr_ethereum', dt_close, cli, conf)

E63 1.3474773412777319
<Worksheet 'total' index:0>


F63 0.5837387210792966
<Worksheet 'total' index:0>


G63 1.4321843187014263
<Worksheet 'total' index:0>


H63 0.5667378372616239
<Worksheet 'total' index:0>


In [41]:
task_total_by_metric('roi_usdt', dt_close, cli, conf)

B64 -7.950394245962078
<Worksheet 'total' index:0>
